<a href="https://www.kaggle.com/code/vincemarcs/mvsa-text-models?scriptVersionId=101310816" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
SEED = 61


import os
import re
import gc
import h5py
import torch
import string
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPooling2D
from keras.layers.core import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score
from keras.initializers import Constant

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [2]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')
    
    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [3]:
def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def load_mvsa_texts(merge=False):
    folder_path = '../input/mvsa-data'
    file_paths = os.listdir(folder_path)
    for path in file_paths:
        file_name = os.path.split(path)[1]
        if file_name.split('.')[1] == 'hdf5':
            if file_name.split('-')[1] == 'single':
                mvsa_single_texts_path = os.path.join(folder_path, path)
            else:
                mvsa_multiple_texts_path = os.path.join(folder_path, path)
    
    mvsa_single = read_hdf5(mvsa_single_texts_path)
    mvsa_multiple = read_hdf5(mvsa_multiple_texts_path)
    
    for x in mvsa_single:
        if x[0] == 'texts':
            mvsa_single_texts = x[1]
            
    for x in mvsa_multiple:
        if x[0] == 'texts':
            mvsa_multiple_texts = x[1]
    
    if merge == True:
        return merge_mvsa(mvsa_single_texts, mvsa_multiple_texts)
    
    return mvsa_single_texts, mvsa_multiple_texts

def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]
        
    return labels, text_labels, image_labels

In [4]:
def clean_text(txt):
#     nonEnglish_regex = re.compile('[^a-zA-Z0-9\\?\\!\\,\\.@#\\+\\-=\\*\'\"><&\\$%\\(\\)\\[\\]:;]+')
#     hashtag_pattern = re.compile('#[a-zA-Z0-9]+')
#     txt_hashtag = re.sub(hashtag_pattern, '', txt)
    at_pattern = re.compile('@[a-zA-Z0-9]+')
    http_pattern = re.compile("((http|ftp|https)://)(([a-zA-Z0-9\._-]+\.[a-zA-Z]{2,6})|([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}))(:[0-9]{1,4})*(/[a-zA-Z0-9\&%_\./-~-]*)?")
    punc_pattern = re.compile('[%s]' % re.escape(string.punctuation))
    txt = re.sub('#', '', txt)
    txt = re.sub(at_pattern, 'user', txt)
    txt = re.sub(http_pattern, 'link', txt)
    txt = re.sub(punc_pattern, '', txt) # only remove punc for word2vec not BERT

    # as the data was crawled using Twitter API, it marked retweet data with RT <user> tag which has no meaning considering it in training
    if txt.startswith('RT user'):
        txt = ''.join(txt.split(':')[1:])    
    txt = txt.strip()
    
    return txt

def get_clean_texts(data):
    cleaned_data = []
    for line in data:
        text = clean_text(line)
        cleaned_data.append(text)
    return cleaned_data

In [5]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 4, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 3)
    plt.plot(history.history['f1_macro'])
    plt.plot(history.history['val_f1_macro'])
    plt.title('Macro F1-SCORE')
    plt.ylabel('f1-macro')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')
    
    fig.add_subplot(1, 4, 4)
    plt.plot(history.history['f1_weighted'])
    plt.plot(history.history['val_f1_weighted'])
    plt.title('Weighted F1-SCORE')
    plt.ylabel('f1-weighted')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [6]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * data.shape[0])
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

In [7]:
def create_model_text(input_shape):
    f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
    f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')
    
    text_input = Input(shape=input_shape)
    dropout = Dropout(DROPOUT_INPUT) (text_input)
    reshape_text = Reshape((1, -1)) (dropout)
    lstm = LSTM(NUM_LSTM) (reshape_text)
    dropout = Dropout(DROPOUT_HIDDEN) (lstm)

    outputs = Dense(NUM_CLASSES, activation='softmax') (dropout)
    
    model = Model(text_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_macro, f1_weighted])
    
    return model

In [8]:
def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))
    
    loss, acc, f1_macro, f1_weighted = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('Macro F1-score:', f1_macro)
        print('Weighted F1-score:', f1_weighted)
        
    return loss, acc, f1_macro, f1_weighted

In [9]:
def run_and_evaluate(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))

#     random_idx = np.random.permutation(len(y))
#     X, y = X[random_idx], y[random_idx]

    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
#     oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
    X_train, y_train = oversample.fit_resample(X_train, y_train)
    
    model = create_model_text(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                        epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                        callbacks=[checkpoint, early_stopping])
    if verbose == 1:
        best_epoch = np.argmin(history.history['val_loss'])
        print('Checkpoint loaded at epoch:', best_epoch)
    
    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [10]:
def run_and_evaluate_w2v(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
#     random_idx = np.random.permutation(len(y))
#     X, y = X[random_idx], y[random_idx]

    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
#     oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
    X_train, y_train = oversample.fit_resample(X_train, y_train)
    
    model = create_model_text_w2v(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                        epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                        callbacks=[checkpoint, early_stopping])
    
    if verbose == 1:
        best_epoch = np.argmin(history.history['val_loss'])
        print('Checkpoint loaded at epoch:', best_epoch)
    
    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [11]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:tomato', axis=0)\
                          .highlight_min(subset=['Loss'], props='color:lawngreen', axis=0)\
                          .highlight_max(subset=['Loss'], props='color:tomato', axis=0)

In [12]:
from IPython.display import display_html

def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

# Load GloVe

In [13]:
MAX_LENGTH = 280 # as in twitter'S word limit
path_to_glove_file = '../input/glovetwitter27b100d/glove.twitter.27B.100d.txt'

In [14]:
mvsa_single_texts, mvsa_multiple_texts = load_mvsa_texts()

mvsa_single_texts = get_clean_texts(mvsa_single_texts)
mvsa_multiple_texts = get_clean_texts(mvsa_multiple_texts)

In [15]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=MAX_LENGTH)

text_ds_single = tf.data.Dataset.from_tensor_slices(mvsa_single_texts).batch(128)
text_ds_multiple = tf.data.Dataset.from_tensor_slices(mvsa_multiple_texts).batch(128)

vectorizer.adapt(text_ds_single)
vectorizer.adapt(text_ds_multiple)

2022-07-20 10:13:15.808622: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-07-20 10:13:16.307173: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [16]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [17]:
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 1193515 word vectors.


In [18]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

embedding_layer_glove = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer= Constant(embedding_matrix),
    trainable=False,
)

def create_model_text_w2v(input_shape):
    f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
    f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')
    
    text_input = Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer_glove(text_input)
    lstm = LSTM(NUM_LSTM) (embedded_sequences)
    lstm = Dropout(DROPOUT_HIDDEN) (lstm)
    outputs = Dense(NUM_CLASSES, activation='softmax') (lstm)
    
    model = Model(text_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_macro, f1_weighted])
    
    return model

Converted 12712 words (7288 misses)


# Load features data

In [19]:
mvsa_single_texts = vectorizer(np.array([[s] for s in mvsa_single_texts])).numpy()
mvsa_multiple_texts = vectorizer(np.array([[s] for s in mvsa_multiple_texts])).numpy()

In [20]:
mvsa_texts = merge_mvsa(mvsa_single_texts, mvsa_multiple_texts)

In [21]:
mvsa_single_bert_base, mvsa_multiple_bert_base = load_mvsa_feature('bert-base')
mvsa_single_pos_bow, mvsa_multiple_pos_bow = load_mvsa_feature('pos-bow')
mvsa_single_pos_tfidf, mvsa_multiple_pos_tfidf = load_mvsa_feature('pos-tfidf')
mvsa_single_ner_bow, mvsa_multiple_ner_bow = load_mvsa_feature('ner-bow')
mvsa_single_ner_tfidf, mvsa_multiple_ner_tfidf = load_mvsa_feature('ner-tfidf')

In [22]:
mvsa_bert_base = merge_mvsa(mvsa_single_bert_base, mvsa_multiple_bert_base)
mvsa_pos_bow = merge_mvsa(mvsa_single_pos_bow, mvsa_multiple_pos_bow)
mvsa_pos_tfidf = merge_mvsa(mvsa_single_pos_tfidf, mvsa_multiple_pos_tfidf)
mvsa_ner_bow = merge_mvsa(mvsa_single_ner_bow, mvsa_multiple_ner_bow)
mvsa_ner_tfidf = merge_mvsa(mvsa_single_ner_tfidf, mvsa_multiple_ner_tfidf)

In [23]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, _ = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, _ = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
# mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

In [24]:
le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3

In [25]:
feature_names = ['glove', 'bert-base', 'bert-pos-bow', 'bert-ner-bow', 'bert-pos-ner-bow', 'bert-pos-tfidf', 'bert-ner-tfidf', 'bert-pos-ner-tfidf']

mvsa_single_features = [mvsa_single_texts,
                        mvsa_single_bert_base, 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_bow), axis=1), 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_ner_bow), axis=1), 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_bow, mvsa_single_ner_bow), axis=1),
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_tfidf), axis=1), 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_ner_tfidf), axis=1), 
                        np.concatenate((mvsa_single_bert_base, mvsa_single_pos_tfidf, mvsa_single_ner_tfidf), axis=1)]

mvsa_multiple_features = [mvsa_multiple_texts,
                          mvsa_multiple_bert_base, 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_bow), axis=1), 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_ner_bow), axis=1), 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_bow, mvsa_multiple_ner_bow), axis=1),
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_tfidf), axis=1), 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_ner_tfidf), axis=1), 
                          np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_tfidf, mvsa_multiple_ner_tfidf), axis=1)]

# mvsa_features = [mvsa_texts,
#                  mvsa_bert_base,
#                  np.concatenate((mvsa_bert_base, mvsa_pos_bow), axis=1), 
#                  np.concatenate((mvsa_bert_base, mvsa_ner_bow), axis=1), 
#                  np.concatenate((mvsa_bert_base, mvsa_pos_bow, mvsa_ner_bow), axis=1),
#                  np.concatenate((mvsa_bert_base, mvsa_pos_tfidf), axis=1), 
#                  np.concatenate((mvsa_bert_base, mvsa_ner_tfidf), axis=1), 
#                  np.concatenate((mvsa_bert_base, mvsa_pos_tfidf, mvsa_ner_tfidf), axis=1)]

In [26]:
def shuffle_mvsa(mvsa_features, labels, indices):
    shuffled_features = []
#     random_idx = np.random.permutation(len(labels))
    for i in range(len(mvsa_features)):
        x = mvsa_features[i][indices]
        shuffled_features.append(x)
    return shuffled_features, labels[indices]

In [27]:
# Fix random indices for consistency between other experiments
mvsa_single_features, mvsa_single_multimodal_labels = shuffle_mvsa(mvsa_single_features, mvsa_single_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-single-shuffle-indices.npy'))
mvsa_multiple_features, mvsa_multiple_multimodal_labels = shuffle_mvsa(mvsa_multiple_features, mvsa_multiple_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-multiple-shuffle-indices.npy'))

# Run models and Evalution display

In [28]:
reset_seeds()
EPOCHS = 100
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1
EARLY_STOPPING = 10
NUM_LSTM = 128
DROPOUT_INPUT = 0.4
DROPOUT_HIDDEN = 0.5

## With original text labels

In [29]:
# print('MVSA-Single: With original text labels\n')
# mvsa_single_histories = []
# mvsa_single_scores = []
# for i in range(len(feature_names)):
#     print('MVSA-Single:', feature_names[i])
#     if feature_names[i] == 'glove':
#         history, score = run_and_evaluate_w2v('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_text_labels, verbose=0)
#     else:
#         history, score = run_and_evaluate('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_text_labels, verbose=0)
#     mvsa_single_histories.append(history)
#     mvsa_single_scores.append(score)
#     print()
# df_single_scores = pd.DataFrame(mvsa_single_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# print('MVSA-Multiple: With original text labels\n')
# mvsa_multiple_histories = []
# mvsa_multiple_scores = []
# for i in range(len(feature_names)):
#     print('MVSA-Multiple:', feature_names[i])
#     if feature_names[i] == 'glove':
#         history, score = run_and_evaluate_w2v('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_text_labels, verbose=0)
#     else:
#         history, score = run_and_evaluate('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_text_labels, verbose=0)
#     mvsa_multiple_histories.append(history)
#     mvsa_multiple_scores.append(score)
#     print()
# df_multiple_scores = pd.DataFrame(mvsa_multiple_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# mvsa_average_scores = np.mean([mvsa_single_scores, mvsa_multiple_scores], axis=0)
# df_average_scores = pd.DataFrame(mvsa_average_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

## With multimodal labels

In [30]:
print('MVSA-Single: With multimodal labels')
mvsa_single_histories2 = []
mvsa_single_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    history, score = run_and_evaluate('single-ML-' + feature_names[i], mvsa_single_features[i], mvsa_single_multimodal_labels, verbose=0)
    mvsa_single_histories2.append(history)
    mvsa_single_scores2.append(score)
    print()
df_single_scores2 = pd.DataFrame(mvsa_single_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('MVSA-Multiple: With multimodal labels')
mvsa_multiple_histories2 = []
mvsa_multiple_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    history, score = run_and_evaluate('multiple-ML-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_multimodal_labels, verbose=0)
    mvsa_multiple_histories2.append(history)
    mvsa_multiple_scores2.append(score)
    print()
df_multiple_scores2 = pd.DataFrame(mvsa_multiple_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

mvsa_average_scores2 = np.mean([mvsa_single_scores2, mvsa_multiple_scores2], axis=0)
df_average_scores2 = pd.DataFrame(mvsa_average_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single: With multimodal labels
MVSA-Single: glove

MVSA-Single: bert-base

MVSA-Single: bert-pos-bow

MVSA-Single: bert-ner-bow

MVSA-Single: bert-pos-ner-bow

MVSA-Single: bert-pos-tfidf

MVSA-Single: bert-ner-tfidf

MVSA-Single: bert-pos-ner-tfidf

MVSA-Multiple: With multimodal labels
MVSA-Multiple: glove

MVSA-Multiple: bert-base

MVSA-Multiple: bert-pos-bow

MVSA-Multiple: bert-ner-bow

MVSA-Multiple: bert-pos-ner-bow

MVSA-Multiple: bert-pos-tfidf

MVSA-Multiple: bert-ner-tfidf

MVSA-Multiple: bert-pos-ner-tfidf



# With merge MVSA data

In [31]:
# print('Both MVSA: With original image labels')
# mvsa_histories3 = []
# mvsa_scores3 = []
# for i in range(len(feature_names)):
#     print('Both MVSA:', feature_names[i])
#     history, score = run_and_evaluate('merge-OL-' + feature_names[i], mvsa_features[i], mvsa_text_labels, verbose=0)
#     mvsa_histories3.append(history)
#     mvsa_scores3.append(score)
#     print()
# df_scores3 = pd.DataFrame(mvsa_scores3, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# print('Both MVSA: With multimodal labels')
# mvsa_histories4 = []
# mvsa_scores4 = []
# for i in range(len(feature_names)):
#     print('Both MVSA:', feature_names[i])
#     history, score = run_and_evaluate('merge-ML-' + feature_names[i], mvsa_features[i], mvsa_multimodal_labels, verbose=0)
#     mvsa_histories4.append(history)
#     mvsa_scores4.append(score)
#     print()
# df_scores4 = pd.DataFrame(mvsa_scores4, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# Display results

In [32]:
# print('With original image labels\n')
# display_dataframes((style_dataframe(df_single_scores), style_dataframe(df_multiple_scores), style_dataframe(df_average_scores)), 
#                    names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

In [33]:
print('With multimodal labels\n')
display_dataframes((style_dataframe(df_single_scores2), style_dataframe(df_multiple_scores2), style_dataframe(df_average_scores2)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With multimodal labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 glove 
 1.122829 
 0.421286 
 0.272687 
 0.386438 
 
 
 bert-base 
 0.812647 
 0.598670 
 0.496873 
 0.611802 
 
 
 bert-pos-bow 
 0.810674 
 0.609756 
 0.505232 
 0.620617 
 
 
 bert-ner-bow 
 0.820776 
 0.609756 
 0.497556 
 0.614071 
 
 
 bert-pos-ner-bow 
 0.819804 
 0.627494 
 0.518737 
 0.628796 
 
 
 bert-pos-tfidf 
 0.815597 
 0.631929 
 0.519891 
 0.638525 
 
 
 bert-ner-tfidf 
 0.795806 
 0.636364 
 0.536301 
 0.644952 
 
 
 bert-pos-ner-tfidf 
 0.825191 
 0.605322 
 0.515399 
 0.617438 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 glove 
 1.118984 
 0.377203 
 0.240356 
 0.397558 
 
 
 bert-base 
 0.777252 
 0.659812 
 0.447664 
 0.606623 
 
 
 bert-pos-bow 
 0.792319 
 0.650999 
 0.441032 
 0.609044 
 
 
 bert-ner-bow 
 0.795095 
 0.653349 
 0.438607 
 0.604364 
 
 
 bert-pos-ner-bow 
 0.803022 
 0.662750 
 0.446125 
 0.619187 
 
 
 bert-pos-tfidf 
 0.786036 
 0.658049 
 0.453748 
 0.616892 
 
 
 bert-ner-tfidf 
 0.781388 
 0.658049 
 0.450488 
 0.612142 
 
 
 bert-pos-ner-tfidf 
 0.791746 
 0.647474 
 0.427743 
 0.593370 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 glove 
 1.120907 
 0.399245 
 0.256521 
 0.391998 
 
 
 bert-base 
 0.794950 
 0.629241 
 0.472268 
 0.609212 
 
 
 bert-pos-bow 
 0.801496 
 0.630377 
 0.473132 
 0.614831 
 
 
 bert-ner-bow 
 0.807936 
 0.631553 
 0.468081 
 0.609217 
 
 
 bert-pos-ner-bow 
 0.811413 
 0.645122 
 0.482431 
 0.623991 
 
 
 bert-pos-tfidf 
 0.800816 
 0.644989 
 0.486819 
 0.627708 
 
 
 bert-ner-tfidf 
 0.788597 
 0.647206 
 0.493395 
 0.628547 
 
 
 bert-pos-ner-tfidf 
 0.808468 
 0.626398 
 0.471571 
 0.605404

In [34]:
# print('With both MVSA merged together\n')
# display_dataframes((style_dataframe(df_scores4)), 
#                    names=['Multimodal labels'])
# style_dataframe(df_scores4)

# Save shuffle indices for other experiments consistency (temporary fix)


In [35]:
# reset_seeds()
# random_idx_single = np.random.permutation(len(mvsa_single_multimodal_labels))
# random_idx_multiple = np.random.permutation(len(mvsa_multiple_multimodal_labels))
# np.save('mvsa-single-shuffle-indices.npy', random_idx_single)
# np.save('mvsa-multiple-shuffle-indices.npy', random_idx_multiple)

In [36]:
# a = np.load('../input/mvsa-shuffle-indices/mvsa-single-shuffle-indices.npy')
# b = np.load('../input/mvsa-shuffle-indices/mvsa-multiple-shuffle-indices.npy')